In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import chart_studio.plotly as py
import plotly.graph_objects as go
import plotly.express as px
import sklearn
from datetime import datetime

In [101]:
lab_ds_path = 'nii_inf/export_LABD_PCR_weekly_201540_201939_FINAL.xlsx'
epi_ds_path = 'nii_inf/export_EPI_MAIN_weekly_201540_201939_FINAL.xlsx'

In [121]:
lab_ds = pd.read_excel(lab_ds_path)
epi_ds = pd.read_excel(epi_ds_path)

In [122]:
main_ds = pd.merge(left=epi_ds, right=lab_ds, how='outer', on=['REGION_NAME', 'DISTRICT_NAME', 'YEAR', 'WEEK'])
main_ds.describe()
# main_ds.columns

Index(['REGION_NAME', 'DISTRICT_NAME', 'LPU_NAME_x', 'YEAR', 'WEEK', 'ARI_0_2',
       'ARI_3_6', 'ARI_7_14', 'ARI_15', 'ARI_TOTAL', 'ILI_0_2', 'ILI_3_6',
       'ILI_7_14', 'ILI_15', 'ILI_TOTAL', 'POP_0_2', 'POP_3_6', 'POP_7_14',
       'POP_15', 'POP_TOTAL', 'LPU_NAME_y', 'SWB_0_2', 'SWB_3_6', 'SWB_7_14',
       'SWB_15', 'SWB_TOTAL', 'POS_0_2', 'POS_3_6', 'POS_7_14', 'POS_15',
       'POS_TOTAL', 'SWBA_0_2', 'SWBA_3_6', 'SWBA_7_14', 'SWBA_15',
       'SWBA_TOTAL', 'POSA_0_2', 'POSA_3_6', 'POSA_7_14', 'POSA_15',
       'POSA_TOTAL'],
      dtype='object')

In [125]:
main_ds

,REGION_NAME,DISTRICT_NAME,LPU_NAME_x,YEAR,WEEK,ARI_0_2,ARI_3_6,ARI_7_14,ARI_15,ARI_TOTAL,...,SWBA_0_2,SWBA_3_6,SWBA_7_14,SWBA_15,SWBA_TOTAL,POSA_0_2,POSA_3_6,POSA_7_14,POSA_15,POSA_TOTAL
0,Дальневосточный ФО,Биробиджан,ЦГиЭ,2015,40,212,163,87,85,547,...,7.0,1.0,3.0,1.0,12.0,5.0,1.0,0.0,0.0,6.0
1,Дальневосточный ФО,Владивосток,ЦГиЭ,2015,40,631,811,359,570,2371,...,4.0,5.0,0.0,22.0,31.0,1.0,1.0,0.0,4.0,6.0
2,Дальневосточный ФО,Магадан,ЦГиЭ,2015,40,149,200,155,128,632,...,11.0,6.0,1.0,4.0,22.0,7.0,2.0,0.0,0.0,9.0
3,Дальневосточный ФО,Петропавловск,ЦГиЭ,2015,40,297,334,226,193,1050,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Дальневосточный ФО,Улан-Удэ,ЦГиЭ,2015,40,144,159,75,228,606,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13018,Южный ФО,Волгоград,ЦГиЭ,2019,39,952,1178,890,1531,4551,...,4.0,3.0,7.0,9.0,23.0,1.0,0.0,2.0,3.0,6.0
13019,Южный ФО,Краснодар,ЦГиЭ,2019,39,45,102,71,415,633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13020,Южный ФО,Ростов-на-Дону,ЦГиЭ,2019,39,697,941,675,920,3233,...,32.0,9.0,7.0,8.0,56.0,25.0,7.0,2.0,0.0,34.0
13021,Южный ФО,Севастополь,ЦГиЭ,2019,39,175,287,402,259,1123,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [123]:
epi_ds.LPU_NAME.unique()

array(['ЦГиЭ'], dtype=object)

In [124]:
lab_ds.LPU_NAME.unique()

array(['ЦГиЭ', 'ФГУЗ', 'ЦЭЭГ', 'НИИ гриппа', 'ГВЦ'], dtype=object)

In [9]:
def get_district_data(ds):
    districts = ds.DISTRICT_NAME.unique()
    data = {}
    for dst in districts:
        dst_df = ds[ds.DISTRICT_NAME==dst].sort_values(['YEAR', 'WEEK'])
        data[dst] = dst_df
    return data

In [103]:
def make_dates(ds):
    ds['DATE'] = ds.apply(lambda x: datetime.strptime(f'{x["YEAR"]}-{x["WEEK"]}-1', "%Y-%W-%w"), axis=1)
    
def normalize(ds, features):
    # Need bold features: ARI_0_2 -> ARI
    all_features = ds.columns
    for feature in features:
        ages = [ft for ft in all_features if ft.startswith(feature)]
        pops = [pop for pop in all_features if pop.startswith('POP')]
        dct = {}
        for age in ages:
            interval = age.replace(feature, '')
            pop = [p for p in pops if p.endswith(interval)][0]
            dct[age] = pop
        for age_feat, age_pop in dct.items():
            ds[f'NORM_{age_feat}'] = ds[age_feat] / ds[age_pop] * 100_000

In [72]:
def plot_timeline(df, features, cities=None):
    if cities == None:
        cities = df.DISTRICT_NAME.unique()
    fig = go.Figure()
    for city in cities:
        city_data = df[df.DISTRICT_NAME==city]
        for feature in features:
            fig.add_trace(go.Scatter(x=city_data.DATE, y=city_data[feature],
                                    name=f'{city} - {feature}'))
    fig.show()

In [106]:
normalize(epi_ds, ['ARI', 'ILI'])
epi_ds

,REGION_NAME,DISTRICT_NAME,LPU_NAME,YEAR,WEEK,ARI_0_2,ARI_3_6,ARI_7_14,ARI_15,ARI_TOTAL,...,NORM_ARI_0_2,NORM_ARI_3_6,NORM_ARI_7_14,NORM_ARI_15,NORM_ARI_TOTAL,NORM_ILI_0_2,NORM_ILI_3_6,NORM_ILI_7_14,NORM_ILI_15,NORM_ILI_TOTAL
0,Дальневосточный ФО,Биробиджан,ЦГиЭ,2015,40,212,163,87,85,547,...,7237.965176,4278.215223,1344.667697,138.058732,731.508351,0.0,0.0,0.0,0.0,0.0
1,Дальневосточный ФО,Владивосток,ЦГиЭ,2015,40,631,811,359,570,2371,...,3213.812774,3115.276764,747.356150,115.088042,402.563093,0.0,0.0,0.0,0.0,0.0
2,Дальневосточный ФО,Магадан,ЦГиЭ,2015,40,149,200,155,128,632,...,4031.385281,4346.881113,1838.889548,151.804457,625.463902,0.0,0.0,0.0,0.0,0.0
3,Дальневосточный ФО,Петропавловск,ЦГиЭ,2015,40,297,334,226,193,1050,...,4397.394137,4185.988219,1627.186983,126.646237,580.062426,0.0,0.0,0.0,0.0,0.0
4,Дальневосточный ФО,Улан-Удэ,ЦГиЭ,2015,40,144,159,75,228,606,...,617.654628,553.409210,164.842411,62.432057,130.959070,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12738,Южный ФО,Волгоград,ЦГиЭ,2019,39,952,1178,890,1531,4551,...,2826.015971,2741.766554,1134.842206,178.343445,449.023367,0.0,0.0,0.0,0.0,0.0
12739,Южный ФО,Краснодар,ЦГиЭ,2019,39,45,102,71,415,633,...,87.545232,199.874588,84.138176,51.656493,63.926286,0.0,0.0,0.0,0.0,0.0
12740,Южный ФО,Ростов-на-Дону,ЦГиЭ,2019,39,697,941,675,920,3233,...,1814.773349,2213.961367,928.741452,95.209907,288.692934,0.0,0.0,0.0,0.0,0.0
12741,Южный ФО,Севастополь,ЦГиЭ,2019,39,175,287,402,259,1123,...,1087.767280,1464.360427,1156.302134,70.723096,257.173609,0.0,0.0,0.0,0.0,0.0


In [127]:
make_dates(epi_ds)
make_dates(lab_ds)
make_dates(main_ds)

In [66]:
dst_lab_data = get_district_data(lab_ds)
dst_epi_data = get_district_data(epi_ds)

In [128]:
plot_timeline(main_ds, features=['POS'])

KeyError: 'POS'

In [71]:
tmp = dst_epi_data['Санкт-Петербург']
# fig = px.line(epi_ds, x='DATE', y='ARI_TOTAL', title='ILI_TOTAL in Cities', color='DISTRICT_NAME')
fig = go.Figure()
fig.add_trace(go.Scatter(x=tmp.DATE, y=tmp.ARI_TOTAL))
# fig = px.line(tmp, x='DATE', y='ARI_TOTAL', title='ILI_TOTAL in Москва')
fig.show()

In [48]:
epi_ds

,REGION_NAME,DISTRICT_NAME,LPU_NAME,YEAR,WEEK,ARI_0_2,ARI_3_6,ARI_7_14,ARI_15,ARI_TOTAL,ILI_0_2,ILI_3_6,ILI_7_14,ILI_15,ILI_TOTAL,POP_0_2,POP_3_6,POP_7_14,POP_15,POP_TOTAL
0,Дальневосточный ФО,Биробиджан,ЦГиЭ,2015,40,212,163,87,85,547,0,0,0,0,0,2929,3810,6470,61568,74777
1,Дальневосточный ФО,Владивосток,ЦГиЭ,2015,40,631,811,359,570,2371,0,0,0,0,0,19634,26033,48036,495273,588976
2,Дальневосточный ФО,Магадан,ЦГиЭ,2015,40,149,200,155,128,632,0,0,0,0,0,3696,4601,8429,84319,101045
3,Дальневосточный ФО,Петропавловск,ЦГиЭ,2015,40,297,334,226,193,1050,0,0,0,0,0,6754,7979,13889,152393,181015
4,Дальневосточный ФО,Улан-Удэ,ЦГиЭ,2015,40,144,159,75,228,606,0,0,0,0,0,23314,28731,45498,365197,462740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12738,Южный ФО,Волгоград,ЦГиЭ,2019,39,952,1178,890,1531,4551,0,0,0,0,0,33687,42965,78425,858456,1013533
12739,Южный ФО,Краснодар,ЦГиЭ,2019,39,45,102,71,415,633,0,0,0,0,0,51402,51032,84385,803384,990203
12740,Южный ФО,Ростов-на-Дону,ЦГиЭ,2019,39,697,941,675,920,3233,0,0,0,0,0,38407,42503,72679,966286,1119875
12741,Южный ФО,Севастополь,ЦГиЭ,2019,39,175,287,402,259,1123,0,0,0,0,0,16088,19599,34766,366217,436670


In [48]:
wk = '2016-2'
dt = datetime.strptime(wk + '-1', "%Y-%W-%w")
print(dt)

2016-01-11 00:00:00
